In [43]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from skimage import io, img_as_float
import itertools
import numpy as np
import random

def load_imgs(set_path):
    imgs = {}
    file = open(set_path + "correct.txt", mode="r")
    lines = file.readlines()
    for i in range(0,len(lines)):
        img = io.imread(set_path + str(i) + ".png")
        img = img_as_float(img)
        imgs[i] = img
    return imgs

    
def score_similarity(pair):
    #TODO compare by better classifier
    img1 = imgs[pair[0]]
    img2 = imgs[pair[1]]
    img1_nz = np.count_nonzero(img1)
    img2_nz = np.count_nonzero(img2)

    return img1_nz - img2_nz

def load_correct_pairs(set_path):
    result = []
    file = open(set_path + "correct.txt", mode="r")
    lines = file.readlines()
    for idx, line in enumerate(lines):
        result.append((idx, int(line)))
    return result


def rank_pairs(pairs):
    scores = []
    for pair in pairs:
        s = score_similarity(pair)
        scores.append((pair, s))
    return sorted(scores, key=lambda x: -x[1])


def score_rank(rank, correct_pair):
    for idx, elem in enumerate(rank):
        if elem[0] == correct_pair:
            return 1/(idx+1)
    return 0


def evaluate_set(set_path):
    print("########### {} ###########".format(set_path))
    correct_pairs = load_correct_pairs(set_path)
    for correct_pair in correct_pairs:
        queried = correct_pair[0]
        candidates = np.delete(np.arange(0,len(correct_pairs)), queried)
        possible_pairs = list(itertools.product([queried],candidates))
        rank = rank_pairs(possible_pairs)
        print("score = {} rank = {} correct = {}".format(score_rank(rank,correct_pair),rank,  correct_pair))    

sets_to_evaluate = ["data/set0/","data/set1/","data/set2/","data/set3/","data/set4/","data/set5/","data/set6/","data/set7/","data/set8/"]
for set_to_eval in sets_to_evaluate:
    imgs = load_imgs(set_to_eval)
    evaluate_set(set_to_eval)





